In [ ]:
from ncclient import manager
import requests
import json

# Method that posts a notification to Webex Teams
def post_notification(notification, bot_token, room_id):
    header = {"Authorization": "Bearer %s" % bot_token,
              "Content-Type": "application/json"}

    payload = {"roomId": room_id,
               "markdown": "```" + notification}

    result = requests.post("https://api.ciscospark.com/v1/messages/",
                           headers=header,
                           data=json.dumps(payload),
                           verify=True)

    if result.status_code == 200:
        print("Successfully posted to Webex Teams room")
    else:
        print("Failed to post to Webex Teams room with status code: %s" % result.status_code)
        if result.status_code == 400:
            print("Check that the room id is correct")
        elif result.status_code == 401:
            print("Check that the bot token is correct")
        elif result.status_code == 404:
            print("Check that the bot is part of the room")


In [ ]:
if __name__ == '__main__':

    # Read NSO credentials
    session = {}
    # NSO credentials
    session['nso'] = {}
    session['nso']['host'] = "198.18.134.50"
    session['nso']['port'] = "2022"
    session['nso']['user'] = "admin"
    session['nso']['pass'] = "admin"

    session['stream'] = "ztp"
    session['teams_bot_token'] = "ZjczZjFmYTctNDkwNy00ZjBlLTljZjQtN2QyNTQ2YzJkMzAwNGMwNzM1OTktNTZl_PF84_1eb65fdf-9643-417f-9974-ad72cae0e10f"
    session['teams_room_id'] = "Y2lzY29zcGFyazovL3VzL1JPT00vNTkxNTljNjAtZmFmOS0xMWU5LTlkZjYtNTFiMmFjN2I5YmY0"
    

    # Connect to NSO
    m = manager.connect(look_for_keys=False,
                         host=session['nso']['host'],
                         port=session['nso']['port'],
                         username=session['nso']['user'],
                         password=session['nso']['pass'],
                         hostkey_verify=False)

    try:
        # Subscribe to notifications of a stream
        m.create_subscription(stream_name=session['stream'])

    except Exception as e:
        print('Failed to create subscription:', e)
    while True:
        # Wait indefinitely for notifications
        notification = m.take_notification()

        # Print notification
        print(notification.notification_xml)

        # Send notification to the Webex Teams room
        post_notification(notification.notification_xml,
                          session['teams_bot_token'],
                          session['teams_room_id'])
    m.close_session()